# Project 6: IMDB

This project involves NLP, decision trees, bagging, boosting, and more!

---

## Load packages

You are likely going to need to install the `imdbpie` package:

    > pip install imdbpie

---

In [1]:
import os
import subprocess
import collections
import re
import csv
import json

import pandas as pd
import numpy as np
import scipy

from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import RandomForestRegressor

import psycopg2
import requests
from imdbpie import Imdb
import nltk

import urllib
from bs4 import BeautifulSoup
import nltk

import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

---

## Part 1: Acquire the Data

You will connect to the IMDB API to query for movies. 

See here for documentation on how to use the package:

https://github.com/richardasaurus/imdb-pie

#### 1. Connect to the IMDB API

In [2]:
imdb = Imdb()
imdb = Imdb(anonymize=True)

#### 2. Query the top 250 rated movies in the database

In [3]:
imdb.top_250()

[{u'can_rate': True,
  u'image': {u'height': 1388,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BODU4MjU4NjIwNl5BMl5BanBnXkFtZTgwMDU2MjEyMDE@._V1_.jpg',
   u'width': 933},
  u'num_votes': 1660365,
  u'rating': 9.3,
  u'tconst': u'tt0111161',
  u'title': u'The Shawshank Redemption',
  u'type': u'feature',
  u'year': u'1994'},
 {u'can_rate': True,
  u'image': {u'height': 500,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BMjEyMjcyNDI4MF5BMl5BanBnXkFtZTcwMDA5Mzg3OA@@._V1_.jpg',
   u'width': 333},
  u'num_votes': 1136449,
  u'rating': 9.2,
  u'tconst': u'tt0068646',
  u'title': u'The Godfather',
  u'type': u'feature',
  u'year': u'1972'},
 {u'can_rate': True,
  u'image': {u'height': 500,
   u'url': u'http://ia.media-imdb.com/images/M/MV5BNDc2NTM3MzU1Nl5BMl5BanBnXkFtZTcwMTA5Mzg3OA@@._V1_.jpg',
   u'width': 333},
  u'num_votes': 776176,
  u'rating': 9,
  u'tconst': u'tt0071562',
  u'title': u'The Godfather: Part II',
  u'type': u'feature',
  u'year': u'1974'},
 {u'can_rate': True,
 

#### 3. Make a dataframe from the movie data

Keep the fields:

    num_votes
    rating
    tconst
    title
    year
    
And discard the rest

In [4]:
movie_data = pd.DataFrame(data = imdb.top_250(), columns = ['num_votes','rating','tconst','title','year'])

In [5]:
movie_data.head()

,num_votes,rating,tconst,title,year
0,1660365,9.3,tt0111161,The Shawshank Redemption,1994
1,1136449,9.2,tt0068646,The Godfather,1972
2,776176,9.0,tt0071562,The Godfather: Part II,1974
3,1646039,9.0,tt0468569,The Dark Knight,2008
4,849441,8.9,tt0108052,Schindler's List,1993


In [6]:
top_100 = movie_data.head(101)

#### 3. Select only the top 100 movies

In [4]:
# movie_data_top_100 = movie_data.sort_values(by=['rating'],ascending=False)
# movie_data_top_100[0:101]

#### 4. Get the genres and runtime for each movie and add them to the dataframe

There can be multiple genres per movie, so this will need some finessing.

In [7]:
import requests
import json

movies = {'genre': [], 'runtime':[]}   # create empty dictionary for genre runtime
for a in top_100.tconst:  # for each IMDB id
    path = "http://www.omdbapi.com/?i=%s&plot=short&r=json" % a   # parse through the paths of each id,
    #substituting the new id via the %s %sub method
    movie_dict = json.loads(requests.get(path).text)   #load the request data into json object
    movies['genre'].append(movie_dict['Genre'])   #append the genres, runtime to a dictionary
    movies['runtime'].append(movie_dict['Runtime'])
    
top_100['genre'] = movies['genre']   # finish dataframe
top_100['runtime'] = movies['runtime'] 

/Users/jrcmalunay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/jrcmalunay/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:13: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [8]:
top_100.head()

,num_votes,rating,tconst,title,year,genre,runtime
0,1660365,9.3,tt0111161,The Shawshank Redemption,1994,"Crime, Drama",142 min
1,1136449,9.2,tt0068646,The Godfather,1972,"Crime, Drama",175 min
2,776176,9.0,tt0071562,The Godfather: Part II,1974,"Crime, Drama",202 min
3,1646039,9.0,tt0468569,The Dark Knight,2008,"Action, Crime, Thriller",152 min
4,849441,8.9,tt0108052,Schindler's List,1993,"Biography, Drama, History",195 min


In [109]:
# import requests
# from bs4 import BeautifulSoup
# from scrapy.selector import Selector
# from scrapy.http import HtmlResponse

# url = 'http://www.omdbapi.com/?'
# response = requests.get(url)

# html = response.text
# html
# Selector(text=html).xpath("//td").extract()

#### 4. Write the Results to a csv

In [9]:
top_100.to_csv('../assets/data/top_100.csv', encoding = 'utf-8')

---

## Part 2: Wrangle the text data

#### 1. Scrape the reviews for the top 100 movies

*Hint*: Use a loop to scrape each page at once

In [12]:
from scrapy.selector import Selector
from scrapy.http import HtmlResponse

imdb_url = "http://www.imdb.com/title/%s/reviews"

movie_reviews = []

review_iteration = range(1, 11)

for x in top_100.tconst:
    new_url = imdb_url % x
    new_response = requests.get(new_url)
    new_HTML = new_response.text
    for i in review_iteration:
        movie_reviews.append(Selector(text=new_HTML).xpath("//div[@id='tn15content']/p[%s]" % i).extract())

In [14]:
movie_reviews[0]

[u'<p>\nWhy do I want to write the 234th comment on The Shawshank Redemption? I\nam not sure - almost everything that could be possibly said about it\nhas been said. But like so many other people who wrote comments, I was\nand am profoundly moved by this simple and eloquent depiction of hope\nand friendship and redemption. <br><br>The only other movie I have ever seen that effects me as strongly is To\nKill a Mockingbird. Both movies leave me feeling cleaner for having\nwatched them.<br><br>I didn\'t intend to see this movie at all: I do not like prison movies\nand I don\'t normally watch them. I work at a branch library and one day\nas I was checking The Shawshank Redemption out to one of our older\npatrons, she said to me, "Whenever I feel down or depressed, I check\nout this movie and watch it and it always makes me feel better." At the\ntime, I thought that was very strange. One day there was nothing on TV\nexcept things I absolutely would not watch under any circumstance or\nthing

In [15]:
# import requests
# from bs4 import BeautifulSoup
# from scrapy.selector import Selector
# from scrapy.http import HtmlResponse

# url = 'http://www.imdb.com/chart/top?ref_=nv_mv_250_6'
# response = requests.get(url)

# html = response.text

# Selector(text=html).xpath("//td[@class='titleColumn']/a/text()").extract()

In [16]:
# top_100_reviews = []
# for review in top_100.tconst:
#     reviews = imdb.get_title_reviews(review, max_results=15)
#     top_100_reviews.append(reviews)

In [17]:
# top_100_reviews[0]

In [18]:
# reviews = {'reviews':[]}
# for i in top_100_reviews:
#     reviews['reviews'].append(i)
# pd.DataFrame(reviews)

# top_100['reviews'] = reviews['reviews']

#### 2. Extract the reviews and the rating per review for each movie

*Note*: "soup" from BeautifulSoup is the html returned from all 25 pages. You'll need to either address each page individually or break them down by elements.

In [21]:
review_ratings = []

rating_iteration = np.linspace(1, 19, 10)

for x in top_100.tconst:
    new_url = imdb_url % x
    new_response = requests.get(new_url)
    new_HTML = new_response.text
    for i in rating_iteration:
        if Selector(text=new_HTML).xpath("//div[@id='tn15content']/div[%s]/img/@alt" % i).extract() == []:
            # If the review has no rating, give it a placeholder of '-1/10'
            review_ratings.append(['-1/10'])
        else:
            review_ratings.append(Selector(text=new_HTML).xpath("//div[@id='tn15content']/div[%s]/img/@alt" % i).extract())

In [ ]:
review_ratings[0]

In [19]:
# top_100_trr = top_100[['title','rating','reviews']]
# top_100_trr.head()

#### 3. Remove the non AlphaNumeric characters from reviews

#### 4. Calculate the top 200 ngrams from the user reviews

Use the `TfidfVectorizer` in sklearn.

Recommended parameters:

    ngram_range = (1, 2)
    stop_words = 'english'
    binary = False
    max_features = 200

In [61]:
from sklearn.feature_extraction.text import TfidfVectorizer

tvec = TfidfVectorizer(stop_words='english')
tvec.fit([, ham])
df  = pd.DataFrame(tvec.transform([spam, ham]).todense(),
                   columns=tvec.get_feature_names(),
                   index=['spam', 'ham'])

df.transpose().sort_values('spam', ascending=False).head(10).transpose()

NameError: name 'spam' is not defined

#### 5. Merge the user reviews and ratings

#### 6. Save this merged dataframe as a csv

---

## Part 3: Combine Tables in PostgreSQL

#### 1. Import your two .csv data files into your Postgre Database as two different tables

For ease, we can call these table1 and table2

#### 2. Connect to database and query the joined set

#### 3. Join the two tables 

#### 4. Select the newly joined table and save two copies of the into dataframes

---

## Part 4: Parsing and Exploratory Data Analysis

#### 1. Rename any columns you think should be renamed for clarity

#### 2. Describe anything interesting or suspicious about your data (quality assurance)

#### 3. Make four visualizations of interest to you using the data

---

## Part 5: Decision Tree Classifiers and Regressors

#### 1. What is our target attribute? 

Choose a target variable for the decision tree regressor and the classifier. 

#### 2. Prepare the X and Y matrices and preprocess data as you see fit

#### 3. Build and cross-validate your decision tree classifier

#### 4. Gridsearch optimal parameters for your classifier. Does the performance improve?

#### 5. Build and cross-validate your decision tree regressor

#### 6. Gridsearch the optimal parameters for your classifier. Does performance improve?

---

## Part 6: Elastic Net


#### 1. Gridsearch optimal parameters for an ElasticNet using the regression target and predictors you used for the decision tree regressor.


#### 2. Is cross-validated performance better or worse than with the decision trees? 

#### 3. Explain why the elastic net may have performed best at that particular l1_ratio and alpha

---

## Part 7: Bagging and Boosting: Random Forests, Extra Trees, and AdaBoost

#### 1. Load the random forest regressor, extra trees regressor, and adaboost regressor from sklearn

#### 2. Gridsearch optimal parameters for the three different ensemble methods.

#### 3. Evaluate the performance of the two bagging and one boosting model. Which performs best?

#### 4. Extract the feature importances from the Random Forest regressor and make a DataFrame pairing variable names with their variable importances.

#### 5. Plot the ranked feature importances.

#### 6.1 [BONUS] Gridsearch an optimal Lasso model and use it for variable selection (make a new predictor matrix with only the variables not zeroed out by the Lasso). 

#### 6.2 [BONUS] Gridsearch your best performing bagging/boosting model from above with the features retained after the Lasso. Does the score improve?

#### 7.1. [BONUS] Select a threshold for variable importance from your Random Forest regressor and use that to perform feature selection, creating a new subset predictor matrix.

#### 7.2 [BONUS] Using BaggingRegressor with a base estimator of your choice, test a model using the feature-selected dataset you made in 7.1

---

## [VERY BONUS] Part 8: PCA

#### 1. Perform a PCA on your predictor matrix

#### 2. Examine the variance explained and determine what components you want to keep based on them.

#### 3. Plot the cumulative variance explained by the ordered principal components.

#### 4. Gridsearch an elastic net using the principal components you selected as your predictors. Does this perform better than the elastic net you fit earlier?

#### 5. Gridsearch a bagging ensemble estimator that you fit before, this time using the principal components as predictors. Does this perform better or worse than the original? 

#### 6. Look at the loadings of the original predictor columns on the first 3 principal components. Is there any kind of intuitive meaning here?

Hint, you will probably want to sort by absolute value of magnitude of loading, and also only look at the obviously important (larger) ones!

# [Extremely Bonus] Part 9:  Clustering

![](https://snag.gy/jPSZ6U.jpg)

 ***Bonus Bonus:***
This extended bonus question is asking to do something we never really talked about but would like for you to attempt based on the assumptions that we learned during this weeks clustering lesson(s).

#### 1. Import your favorite clustering module

#### 2. Encode categoricals

#### 3. Evaluate cluster metics solely based on a range of K
If K-Means:  SSE/Inertia vs Silhouette (ie: Elbow), silhouette average, etc

#### 4.  Look at your data based on the subset of your predicted clusters.
Assign the cluster predictions back to your dataframe in order to see them in context.  This is great to be able to group by cluster to get a sense of the data that clumped together.

#### 5. Describe your findings based on the predicted clusters 
_How well did it do?  What's good or bad?  How would you improve this? Does any of it make sense?_